# Serap YALÇIN
NLP Course Project 
29th March 2020 

In [1]:
import numpy as np
import pandas as pd
import re
import json
from pprint import pprint
from snowballstemmer import TurkishStemmer
import nltk
#nltk.download("stopwords")
from nltk.util import ngrams
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures

# 1- Read the dataset and parse it (i.e. tokenization, stemming, stopwords removal). (10+10= 20% of the assignment)


In [2]:
##READ the Datasets 
data=pd.read_excel(r"C:\Users\budaks\Desktop\YL 2020 Bahar\NLP\MidtermProject\cv_listesi.xlsx")
data.columns = ['No', 'RefN','Resume','Yetenekler','ResumeAll']
##ResumeAll includes all of the information coming from other columns
data = data[['No','RefN','ResumeAll']]
data['ResumeAll']=data['ResumeAll'].str.lower()

data.head()

,No,RefN,ResumeAll
0,0,473,- mapco ve turkcellgroup bünyesindeki şirketle...
1,1,900,<p>aile şirketimizde ve inşaat sektöründe paza...
2,2,1323,"bilgi i̇şlem yöneticisi.&nbsp;moda tekstil, ro..."
3,3,3748,- setting up departments in the production fac...
4,4,6363,<p>i̇ç satın alma faaliyetlerinden sorumlu sat...


In [3]:
len(data)

8012

In [4]:
#Read Ads
adsdata=pd.read_excel(r"C:\Users\budaks\Desktop\YL 2020 Bahar\NLP\MidtermProject\ilan_içerikleri.xlsx")
adsdata.columns = ['AdsNo', 'JobId','Ilan']
adsdata['Ilan']=adsdata['Ilan'].str.lower()
adsdata.head()

,AdsNo,JobId,Ilan
0,0,1870838,makam şoförü \r\n<ul>\r\n<li>en az 5 yıl makam...
1,1,1676048,şoför / sürücü avrupa bölge müdürlüğü'ne bağlı...
2,2,2141993,şoför / sürücü sitil çizgi şirketler grubu bün...
3,3,2175311,şoför / sürücü <strong>alfred concierge hizmet...
4,4,2193892,şoför / sürücü <ul><li><span><span>yoğun iş te...


In [5]:
##Create functions to remove punctuation, tokenize, remove stopwords, and stem
import re 
  
def remove(string): 
    pattern = re.compile(r'[,.;@#?!\&$!"#$%&()*+, -./\':;<=>?@[\]^_`{|}~]+\ *') 
    
    return re.sub(pattern, ' ', string)

In [6]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

In [7]:
##Create functions to remove punctuation, tokenize, remove stopwords, and stem
turkish_stops=set(stopwords.words("turkish"))
stopwords = set(stopwords.words('english'))
def clean_text(statement):
    
    tokenizer = RegexpTokenizer(r"\w+")
    statement_tokens = tokenizer.tokenize(str(statement))

   ## ps = nltk.PorterStemmer()
    TurkStem=TurkishStemmer()
    
    statements_clean = []     

    for word in statement_tokens:
        if word not in turkish_stops:
            if word not in stopwords:
                stem_word = TurkStem.stemWord(word)
                statements_clean.append(stem_word)
    return statements_clean

In [8]:
data['ResumeAllClean'] = data['ResumeAll'].apply(lambda x: remove(x))
adsdata['IlanClean'] = adsdata['Ilan'].apply(lambda x: remove(x))

In [9]:
data.head()

,No,RefN,ResumeAll,ResumeAllClean
0,0,473,- mapco ve turkcellgroup bünyesindeki şirketle...,mapco ve turkcellgroup bünyesindeki şirketler...
1,1,900,<p>aile şirketimizde ve inşaat sektöründe paza...,p aile şirketimizde ve inşaat sektöründe paza...
2,2,1323,"bilgi i̇şlem yöneticisi.&nbsp;moda tekstil, ro...",bilgi i̇şlem yöneticisi nbsp moda tekstil rote...
3,3,3748,- setting up departments in the production fac...,setting up departments in the production faci...
4,4,6363,<p>i̇ç satın alma faaliyetlerinden sorumlu sat...,p i̇ç satın alma faaliyetlerinden sorumlu sat...


In [10]:
adsdata.head()

,AdsNo,JobId,Ilan,IlanClean
0,0,1870838,makam şoförü \r\n<ul>\r\n<li>en az 5 yıl makam...,makam şoförü \r\n ul \r\n li en az 5 yıl makam...
1,1,1676048,şoför / sürücü avrupa bölge müdürlüğü'ne bağlı...,şoför sürücü avrupa bölge müdürlüğü ne bağlı ş...
2,2,2141993,şoför / sürücü sitil çizgi şirketler grubu bün...,şoför sürücü sitil çizgi şirketler grubu bünye...
3,3,2175311,şoför / sürücü <strong>alfred concierge hizmet...,şoför sürücü strong alfred concierge hizmetler...
4,4,2193892,şoför / sürücü <ul><li><span><span>yoğun iş te...,şoför sürücü ul li span span yoğun iş temposun...


In [11]:
data["ResumeAllClean"]=data["ResumeAllClean"].map(clean_text)
adsdata["IlanClean"]=adsdata["IlanClean"].map(clean_text)

In [12]:
data.head()

,No,RefN,ResumeAll,ResumeAllClean
0,0,473,- mapco ve turkcellgroup bünyesindeki şirketle...,"[mapco, turkcellgroup, bünyes, şirket, ms, bac..."
1,1,900,<p>aile şirketimizde ve inşaat sektöründe paza...,"[p, ai, şirket, inşaat, sektör, pazarla, satış..."
2,2,1323,"bilgi i̇şlem yöneticisi.&nbsp;moda tekstil, ro...","[bilgi, şlem, yöneticis, nbsp, mo, tekstil, ro..."
3,3,3748,- setting up departments in the production fac...,"[settingi, departments, production, facility, ..."
4,4,6363,<p>i̇ç satın alma faaliyetlerinden sorumlu sat...,"[p, ç, sat, alma, faaliyet, sorumlu, sat, alma..."


In [13]:
adsdata.head()

,AdsNo,JobId,Ilan,IlanClean
0,0,1870838,makam şoförü \r\n<ul>\r\n<li>en az 5 yıl makam...,"[maka, şoför, ul, li, 5, yıl, maka, şoförlük, ..."
1,1,1676048,şoför / sürücü avrupa bölge müdürlüğü'ne bağlı...,"[şoför, sürüç, avrup, bölge, müdürlük, bağlı, ..."
2,2,2141993,şoför / sürücü sitil çizgi şirketler grubu bün...,"[şoför, sürüç, sitil, çizgi, şirket, grup, bün..."
3,3,2175311,şoför / sürücü <strong>alfred concierge hizmet...,"[şoför, sürüç, strong, alfredi, concierge, hiz..."
4,4,2193892,şoför / sürücü <ul><li><span><span>yoğun iş te...,"[şoför, sürüç, ul, li, span, span, yok, iş, te..."


In [14]:
data.ResumeAllClean[1]

['p',
 'ai',
 'şirket',
 'inşaat',
 'sektör',
 'pazarla',
 'satış',
 'destek',
 'birimlerin',
 'satış',
 'strateji',
 'finansma',
 'siste',
 'entegrasyo',
 'şirke',
 'ait',
 'izmir',
 'ankar',
 'stanbul',
 'teknoloj',
 'alt',
 'yapı',
 'oluşturulmas',
 'iş',
 'strateji',
 'hedef',
 'doğrultu',
 'belirlene',
 'finans',
 'nsan',
 'kaynak',
 'sat',
 'alma',
 'bak',
 'kali',
 'müşter',
 'hizmet',
 'ait',
 'kurumsal',
 'iş',
 'uygulamas',
 'ihtiyaç',
 'analiz',
 'edilmes',
 'iş',
 'çözüm',
 'belirlenmes',
 'p',
 'bilgisayar',
 'programcılık',
 'mezuniyet',
 'dona',
 'onar',
 'teknik',
 'destek',
 'eğitim',
 'hilto',
 'universty',
 'management',
 'levels',
 'trainingi',
 'sertifika',
 'şletme',
 'bilgiişle',
 'entegrasyo',
 'konu',
 'uluslararas',
 'ulusal',
 'çap',
 'firma',
 'geniş',
 'yok',
 'süreçli',
 'iş',
 'tecrübes',
 '25',
 'yıllık',
 'teknik',
 'destek',
 'yazıl',
 'tecrübes',
 'p',
 'hilto',
 'standart',
 'kur',
 'dijital',
 'dönüş',
 'süreç',
 'liderlik',
 'edilmes',
 'holdingi',

In [15]:
data['ResumeAllClean2'] = data['ResumeAllClean']
adsdata['IlanClean2'] = adsdata['IlanClean']

In [16]:

for i in data.ResumeAllClean.index:
    data.ResumeAllClean2[i]=' '.join(data.ResumeAllClean[i])
    
##data.ResumeAllClean2[0]

C:\Users\budaks\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [17]:
for i in adsdata.IlanClean.index:
    adsdata.IlanClean2[i]=' '.join(adsdata.IlanClean[i])
    


C:\Users\budaks\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [18]:
adsdata.IlanClean2[0]

'maka şoför ul li 5 yıl maka şoförlük tecrübe sahip li li diksiyo düzgü iletiş beceris yük li li dış görünüm öze göstere li li  sürüş teknik konu bilgil deneyimli li li b sınıf sürüç belge sahip aktif otomobil kullanabile li li esnek çalışma saat u sağlayabilecek li li askerlik hizmet tamamlamış li li strong anadol yaka strong tercihe suadi nbsp ikamet e li li referans verebilecek li ul br br h3 ş tan h3 ul li bağlı bulunduk yönetiç taraf şirket iç dış verile görev sorumluluk üstlenebilecek li li tahsis edile araç trafik kural uygu olarak kullanım öze gösterecek li li kullanmak olduk araç periyodik bakım zaman eksiksiz bir şekil yer getirilme sağlamak li li kişisel hijye temizlik kural uyarak belirtile standart çerçeve hizmet vermek li ul'

In [19]:
data.ResumeAllClean2[0]

'mapco turkcellgroup bünyes şirket ms backoffiç server w2k exchange isa sql client kurul bak tamir 150 civar pc 30 server çevre birim yazıç scanner vs br pc cevre birim ayak tutulmas kullanıcı destek verilmes br lan wan kontrolü network ha raporlamas performans iyileştirmes br inet mail dış network bağlantı ayak tutulmas superonl turkcell da center buluna microsoft intel microsoft backoffiç small backoffiç ms offiç windows islet sistem server client network kablola server access linux proxy firewall client hp dell 2003 windows işlet 2002 activ directory exchange isa cisco cihaz wireless network adsl microsoft backoffiç small backoffiç ms offiç windows islet sistem server client network kablola server access linux proxy firewall client hp dell 2003 windows işlet 2002 activ directory exchange isa cisco cihaz wireless network adsl p 11 hastane 3500 pc 100 sunuç 200 switch firewall diğer siste network ekiplan 30 bilgi sistem personel sorumluluk p p sahip olduk süreç performans ölçümlemek t

In [20]:
data.ResumeAllClean2.str.split(expand=True).stack().value_counts()


p                    87698
li                   33137
br                   30891
olarak               19516
yönet                18661
offiç                18418
strong               17857
satış                16445
iş                   16112
bir                  15790
program              15045
nbsp                 14825
ms                   14759
takip                14651
span                 14020
word                 13653
microsoft            13345
çalışma              13227
excel                13127
proje                12049
firma                11840
windows              11077
yıl                   9814
müşter                9647
görev                 9646
süreç                 8786
bilgi                 8566
ş                     8531
style                 8416
eğit                  8242
                     ...  
europe                   1
ruhsatlandırmak          1
çı                       1
kokil                    1
zarama                   1
ningbo                   1
h

In [21]:
adsdata.IlanClean2.str.split(expand=True).stack().value_counts()

li              19249
span            10342
nbsp             9476
x                8521
p                6356
br               5420
strong           3920
ul               2714
h3               1680
sahip            1277
satış            1247
çalışma          1237
ş                1181
müşter            981
ola               921
em                881
class             854
tan               848
mez               824
hizmet            774
tercihe           761
u                 742
bir               724
olarak            719
konu              714
yıl               714
font              642
takip             622
msonormal         607
iş                585
                ...  
zürih               1
nün                 1
ema                 1
üstlenebile         1
fil                 1
tamamala            1
trial               1
begeistert          1
audiences           1
hikâye              1
turnover            1
şablon              1
sever               1
akyaz               1
vardiyel  

In [22]:
# Remove unwanted word to get better running performance 
unwantedwords = ['ol','yap','bir','p','li','br','nbsp','strong','n','al','ş','ye','span','olarak']


word_tokens = data['ResumeAllClean']

for i in data.ResumeAllClean.index:
    word_tokens = data.ResumeAllClean[i]
    removing_stopwords = [word for word in word_tokens if word not in unwantedwords]
    data.ResumeAllClean[i] = removing_stopwords


C:\Users\budaks\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [23]:
data.head()

,No,RefN,ResumeAll,ResumeAllClean,ResumeAllClean2
0,0,473,- mapco ve turkcellgroup bünyesindeki şirketle...,"[mapco, turkcellgroup, bünyes, şirket, ms, bac...",mapco turkcellgroup bünyes şirket ms backoffiç...
1,1,900,<p>aile şirketimizde ve inşaat sektöründe paza...,"[ai, şirket, inşaat, sektör, pazarla, satış, d...",p ai şirket inşaat sektör pazarla satış destek...
2,2,1323,"bilgi i̇şlem yöneticisi.&nbsp;moda tekstil, ro...","[bilgi, şlem, yöneticis, mo, tekstil, roteks, ...",bilgi şlem yöneticis nbsp mo tekstil roteks te...
3,3,3748,- setting up departments in the production fac...,"[settingi, departments, production, facility, ...",settingi departments production facility br or...
4,4,6363,<p>i̇ç satın alma faaliyetlerinden sorumlu sat...,"[ç, sat, alma, faaliyet, sorumlu, sat, alma, m...",p ç sat alma faaliyet sorumlu sat alma müdür o...


In [24]:
# Remove unwanted word to get better running performance 
unwantedwordsforads = ['li','span','p','x','br','nbsp','strong','n','al','ş','p','lu','h3','lu','bir','u','tan','mez','font','em']


word_tokensads = adsdata['IlanClean']

for i in adsdata.IlanClean.index:
    word_tokensads = adsdata.IlanClean[i]
    removing_stopwords = [word for word in word_tokensads if word not in unwantedwordsforads]
    adsdata.IlanClean[i] = removing_stopwords



C:\Users\budaks\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [25]:
adsdata.head()

,AdsNo,JobId,Ilan,IlanClean,IlanClean2
0,0,1870838,makam şoförü \r\n<ul>\r\n<li>en az 5 yıl makam...,"[maka, şoför, ul, 5, yıl, maka, şoförlük, tecr...",maka şoför ul li 5 yıl maka şoförlük tecrübe s...
1,1,1676048,şoför / sürücü avrupa bölge müdürlüğü'ne bağlı...,"[şoför, sürüç, avrup, bölge, müdürlük, bağlı, ...",şoför sürüç avrup bölge müdürlük bağlı şube de...
2,2,2141993,şoför / sürücü sitil çizgi şirketler grubu bün...,"[şoför, sürüç, sitil, çizgi, şirket, grup, bün...",şoför sürüç sitil çizgi şirket grup bünye cent...
3,3,2175311,şoför / sürücü <strong>alfred concierge hizmet...,"[şoför, sürüç, alfredi, concierge, hizmet, tür...",şoför sürüç strong alfredi concierge hizmet st...
4,4,2193892,şoför / sürücü <ul><li><span><span>yoğun iş te...,"[şoför, sürüç, ul, yok, iş, tempo, esnek, çalı...",şoför sürüç ul li span span yok iş tempo esnek...


# 2- for a CV (cv_listesi.xlsx), encode it using  count vectorizer (required) (10%), then tfidf vectorizer(required) (15%), then RNNs (2%, bonus) document2vec (bonus, 2%). (25% of the assignment)

In [26]:
#Count Vectorizer
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
X_counts = count_vect.fit_transform(data['ResumeAllClean2'])
print(X_counts.shape)
print(count_vect.get_feature_names())

(8012, 60513)
['00', '000', '0000', '000000', '000000pt', '0000132322313', '000400618300013', '000415166000066', '000dk', '000k', '000m2', '000metrekare', '000m²', '000sku', '000tl', '000usd', '001', '002060', '0049', '0049172', '005', '0090', '01', '012', '015', '0161611', '01e', '02', '0212', '0216', '0232', '02386', '02473', '02564', '02613', '0266768598558', '02675', '027', '02711', '02751', '03', '03016', '03062', '03067', '03223551500', '034', '04', '04001034259807', '0464', '0465', '0466', '0467', '05', '0506', '05060531771', '05068536363', '0507', '05076056221', '05077636103', '053', '0530', '05302398285', '05303484918', '05304078179', '05309266545', '0531', '05313797256', '0532', '05322020025', '05322667857', '05324062416', '0533', '05334288231', '05336323967', '05336448339', '05342269139', '0535', '0536', '0537', '05374664066', '05376459397', '05383550569', '05396680036', '05399310301', '0542', '05422876802', '0543', '0545', '05464351266', '05466968196', '05510', '05519405559

In [27]:
print(X_counts.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [28]:
# Tf-Idf
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='utf-8', ngram_range=(1, 3), stop_words=turkish_stops)
features = tfidf.fit_transform(data.ResumeAllClean2).toarray()
features.shape

(8012, 62557)

In [29]:
print(features)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


# 3- For a job ad (ilan_icerikleri.xlsx), encode it using  count vectorizer (required) (10%), then tfidf vectorizer(required) (15%), then RNNs (2%, bonus) document2vec (bonus, 2%). (25% of the assignment)

In [30]:
#Count Vectorizer
from sklearn.feature_extraction.text import CountVectorizer

count_vect2 = CountVectorizer()
Y_counts = count_vect2.fit_transform(adsdata['IlanClean2'])

In [31]:
print(Y_counts.shape)
print(count_vect2.get_feature_names())

(1000, 11400)
['00', '000', '000000', '000in', '000m2', '01', '02', '020', '03a294cc', '05', '07', '08', '09', '0d0d0d', '0pt', '10', '100', '1000', '1000mbps', '105', '10649', '108', '10850', '11', '110', '112', '115', '116', '119', '11pt', '12', '120', '1200', '12000', '125', '1250', '127', '128', '12px', '13', '130', '14', '140', '14000', '14001', '1480', '14px', '15', '150', '1500', '16', '160', '16949', '17', '170', '17021', '17025', '17065', '18', '180', '1800', '186', '1885', '1892', '19', '1903', '1906', '1950', '1953', '1956', '1958', '1964', '1968', '1969', '1971', '1973', '1974', '1976', '1977', '1980', '1982', '1983', '1984', '1985', '1987', '1987den', '1989', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '1yıl', '20', '200', '2000', '2000007629395px', '2000tl', '2001', '2002', '2003', '2005', '2006', '2007', '2008', '2009', '200tl', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '202', '2020', '2021', '2023', '21',

In [32]:
print(Y_counts.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 2 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [4 0 0 ... 0 0 0]]


In [33]:
# Tf-Idf
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='utf-8', ngram_range=(1, 3), stop_words=turkish_stops)
featuresads = tfidf.fit_transform(adsdata.IlanClean2).toarray()
featuresads.shape

(1000, 9899)

In [34]:
print(featuresads)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.1284868  0.07082038 0.         ... 0.         0.         0.        ]]


# 1st PART END

# 2nd PART of ASSIGNMENT

In [35]:
##READ the label Datasets 
datalabel=pd.read_excel(r"C:\Users\budaks\Desktop\YL 2020 Bahar\NLP\MidtermProject\ad_cv_evaluation_results_simplified.xlsx")
##data.columns = ['RowId', 'JobId','CvId','Label']

datalabel.head()

,Unnamed: 0,JobId,CvId,Label
0,0,490619,11702287,1
1,1,490619,100819267,1
2,2,490619,105843622,1
3,3,490619,106155661,1
4,4,490619,108241683,1


In [36]:
##X_tfidf_feat = pd.concat([data['TermId'], data['Name'],data['ModuleId'],data['Type'],data['BaseTermId'],data['Sensitivity'],data['IsPrivateData'], pd.DataFrame(features)], axis=1)
Count_feat = pd.concat([data['RefN'], pd.DataFrame(X_counts.toarray())], axis=1)

In [37]:
Count_feat.head()

,RefN,0,1,2,3,4,5,6,7,8,...,60503,60504,60505,60506,60507,60508,60509,60510,60511,60512
0,473,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,900,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1323,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3748,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6363,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
##X_tfidf_feat = pd.concat([data['TermId'], data['Name'],data['ModuleId'],data['Type'],data['BaseTermId'],data['Sensitivity'],data['IsPrivateData'], pd.DataFrame(features)], axis=1)
Count_featads = pd.concat([ adsdata['JobId'], pd.DataFrame(Y_counts.toarray())], axis=1)

In [39]:
Count_featads.head()

,JobId,0,1,2,3,4,5,6,7,8,...,11390,11391,11392,11393,11394,11395,11396,11397,11398,11399
0,1870838,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1676048,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2141993,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2175311,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2193892,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
Count_feat=Count_feat.add_prefix('cv_')

In [41]:
Count_featads=Count_featads.add_prefix('ads_')

In [42]:
alldata=datalabel.join(Count_featads.set_index('ads_JobId'), on ='JobId')

In [43]:
alldata=alldata.join(Count_feat.set_index('cv_RefN'), on ='CvId')

In [44]:
alldata.head()

,Unnamed: 0,JobId,CvId,Label,ads_0,ads_1,ads_2,ads_3,ads_4,ads_5,...,cv_60503,cv_60504,cv_60505,cv_60506,cv_60507,cv_60508,cv_60509,cv_60510,cv_60511,cv_60512
0,0,490619,11702287,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,490619,100819267,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,490619,105843622,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,490619,106155661,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,490619,108241683,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Random Forest Classifier 
for each encoding, feed it to random forest and try to predict the labels in (ilan_cv_degerlendirme.xlsx) and report the accuracy. (30% of the assignment)

In [45]:
##labels
label = alldata['Label']

In [48]:
##features
x =  alldata.drop(['Unnamed: 0','Label','JobId','CvId'], axis=1)

In [49]:
x.head()

,ads_0,ads_1,ads_2,ads_3,ads_4,ads_5,ads_6,ads_7,ads_8,ads_9,...,cv_60503,cv_60504,cv_60505,cv_60506,cv_60507,cv_60508,cv_60509,cv_60510,cv_60511,cv_60512
0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
x.to_excel(r"C:\Users\budaks\Desktop\YL 2020 Bahar\NLP\Project\CVEvaluationOutputX.xlsx", index = False)

MemoryError: 

In [ ]:
y.to_excel(r"C:\Users\budaks\Desktop\YL 2020 Bahar\NLP\Project\CVEvaluationOutputY.xlsx", index = False)

In [54]:
#split data as train and test
from sklearn.model_selection import train_test_split


In [61]:
xdnm = x[:100]
ydnm = y[:100]

NameError: name 'y' is not defined

In [62]:
len(ydnm)

NameError: name 'ydnm' is not defined

In [55]:

X_train, X_test, y_train, y_test = train_test_split(x, label, test_size=0.3)

MemoryError: 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
rf.fit(X_train,y_train)

y_pred=rf.predict(X_test)

In [ ]:
from sklearn import metrics

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


In [ ]:
##rf = RandomForestClassifier()
##param = {'n_estimators': [10, 150, 300],
##        'max_depth': [30, 60, 90, None]}

##gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
##gs_fit = gs.fit(alldata, alldata['label'])
##pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]